In [1]:
#from importlib import reload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize']=(10,18)
%matplotlib inline
from datetime import datetime
from datetime import date
#import xgboost as xgb
from sklearn.cluster import MiniBatchKMeans
import seaborn as sns # plot beautiful charts
import warnings
import math
sns.set()
warnings.filterwarnings('ignore')

In [2]:
data_train = pd.read_csv('input/train.csv').drop(columns=['dropoff_datetime'])
data_test = pd.read_csv('input/test.csv')

#data_train.head()
#. Add fastest routes and merge them into the origial dataframe
data_train_fastroute1 = pd.read_csv('input/fastest_routes_train_part_1.csv',
                        usecols=['id', 'total_distance','number_of_steps',  'step_location_list'])
data_train_fastroute2 = pd.read_csv('input/fastest_routes_train_part_2.csv',
                        usecols=['id', 'total_distance','number_of_steps',  'step_location_list'])
data_train_fastroute = pd.concat((data_train_fastroute1,data_train_fastroute2))
data_train = data_train.join(data_train_fastroute.set_index('id'), on='id')
data_train.head()
# Clean rows that have null values
data_train = data_train.dropna()

In [5]:
##. Creating malicious trips by adding delta values (assuming each follows uniform distribution) 
#.  to routes for each trip

from statistics import median 
from keras.preprocessing.sequence import pad_sequences
delta_max_trj = 2.0
#. Assign a new label column for representing whether the trip info is malicious or not
malicious_trip = []
#data_train['trip_duration'] = data_train['trip_duration'].apply(lambda x: x + random.uniform(0,x/5))
#data_train['total_distance'] = data_train['total_distance'].apply(lambda x: x + random.uniform(0,x/5))

#. For step_location_list in each vehicle's trip,
#     a). Extract a two dimentional list from trajecotry while each element is a GPS location
trj_list = [] #  A list of lists each of which contains trajectory of the corresponding vehicle

for index, row in data_train.iterrows():
    flag_malicious = False
    if(np.random.normal(0,1) < 0.3):
        flag_malicious = True #. the row in current iteration corresponds to a malicious trip
        malicious_trip.append(1)
        row['trip_duration'] += np.random.uniform(0,float(row['trip_duration'])/5)
        row['total_distance'] += np.random.uniform(0,float(row['total_distance'])/5)
    else:
        malicious_trip.append(0)
            
    route = row['step_location_list']
    delta_dist = float(row['total_distance'])/5
    trj = route.split('|')
    trj.pop()   #. remove pickup location as it is already in dataframe
    trj.pop(0)  #. remove dropoff location
    gps_list = []
    #step_index = 0
    for gps_loc in trj:
        gps_log_lat_string = gps_loc.split(',')
        gps_log_lat = [float(element) for element in gps_log_lat_string]
        if flag_malicious == True:
            gps_log_lat[0] += np.random.uniform(-delta_dist,delta_dist)
            gps_log_lat[1] += np.random.uniform(-delta_dist,delta_dist)
        gps_list.append([gps_log_lat[0],gps_log_lat[1]])
        #step_index++
    trj_list.append(gps_list)
 
    
#.     b). Use pre-padding to pad each list to the same length. The max num of steps is 10
#col_num_step = data_train['number_of_steps'].values
#print(median(col_num_step))  # 6
#print(max(col_num_step))     #. 46 maximum num of steps, pretty long
pad_sequences(trj_list, maxlen = 10, dtype='float32', padding='pre')

array([[[    0.      ,     0.      ],
        [    0.      ,     0.      ],
        [    0.      ,     0.      ],
        ...,
        [  -73.981995,    40.76769 ],
        [  -73.979195,    40.771782],
        [  -73.97028 ,    40.768024]],

       [[    0.      ,     0.      ],
        [    0.      ,     0.      ],
        [    0.      ,     0.      ],
        ...,
        [   65.874084,  -375.56058 ],
        [ -367.34457 ,  -320.58466 ],
        [ -491.52728 ,  -449.11298 ]],

       [[-2099.236   ,   166.77582 ],
        [ 2161.2573  , -1212.8766  ],
        [-2305.2292  ,  -448.13348 ],
        ...,
        [  363.0345  ,   467.65295 ],
        [  826.4124  ,  2362.3713  ],
        [-1709.385   ,  2472.826   ]],

       ...,

       [[    0.      ,     0.      ],
        [    0.      ,     0.      ],
        [  -73.95374 ,    40.766857],
        ...,
        [  -73.999306,    40.708496],
        [  -74.0007  ,    40.709885],
        [  -74.001114,    40.710194]],

       [[    0.

In [6]:
#.    c.) iterating trj_list, and construct dataframe colums
trj_col_lists = [[] for _ in range(20)]
for current_trj in trj_list:
    for j in range(10):
        trj_col_lists[2*j].append(current_trj[j][0])
        trj_col_lists[2*j+1].append(current_trj[j][1])

trj_features = pd.DataFrame(index = range(len(data_train)))

for index_trj_col, trj_col in enumerate(trj_col_lists, start=1):
    trj_column_name = None
    if (index_trj_col%2) != 0: # longitude
        trj_column_name = "trj_log{}".format(math.ceil(index_trj_col/2))
    else:
        trj_column_name = "trj_lat{}".format(math.ceil(index_trj_col/2))
    trj_features[trj_column_name] = trj_col
    


IndexError: list index out of range

In [ ]:
#. putting other features in a ndarray with trajectory list, and assign y as labels